In [1]:
#@title Define if we are on Colab and mount drive { display-mode: "form" }
try:
  from google.colab import drive
  drive.mount('/content/gdrive')
  IN_COLAB = True
except:
  IN_COLAB = False

In [2]:
#@title (COLAB ONLY) Clone GitHub repo { display-mode: "form" }

if IN_COLAB:
  !git clone https://github.com/lluissalord/radiology_ai.git

  %cd radiology_ai

In [3]:
#@title (COLAB ONLY) Code for setting the environment { display-mode: "form" }
%%capture
if IN_COLAB:
  PATH_DRIVER = '/content/gdrive/My Drive/'
  DATA_FOLDER = 'Dataset/'
  RAW_PATH = ''
  TMP_PREFIX = '/tmp/'
else:
  PATH_DRIVER = ''
  DATA_FOLDER = 'data/'
  RAW_PATH = 'E:/Google Drive/'
  TMP_PREFIX = 'data/tmp/'

import os

filename_prefix = 'IMG_'

PATH_PREFIX = os.path.join(PATH_DRIVER, DATA_FOLDER, '')

RAW_PREFIX = os.path.join(PATH_DRIVER, RAW_PATH, '')
raw_folder = RAW_PREFIX + 'DICOMS'

ORGANIZE_FOLDER = PATH_PREFIX + 'pending_classification'

PREPROCESS_FOLDER = PATH_PREFIX + 'preprocess'
RAW_PREPROCESS_FOLDER = PATH_PREFIX + 'raw_preprocess'

DOUBT_FOLDER = TMP_PREFIX + 'doubt'
TRAIN_FOLDER = TMP_PREFIX + 'train'

MODELS_FOLDER = os.path.join(PATH_PREFIX ,'models')

KNEE_SVM_MODEL_PATH = PATH_PREFIX + 'extras/svm_model.npy'

In [4]:
import os

In [ ]:
!pip install pydicom

In [5]:
from utils.organize import *

groups = ['Daniel_Berlanga', 'David_Codina', 'Federico_Yanez', 'Guillem_Navarro', 'Jose_Colombo', 'Raul_Franco', 'Hugo_Munoz', 'Tomas_Maio']
subgroup_length = 20
relation_filepath = PATH_PREFIX + 'relation.csv'
filename_prefix = 'IMG_'
copy = True

able_overwrite = False

NUM_SAMPLES_ORGANISE = 6 * subgroup_length * len(groups)

In [6]:
metadata_labels_path = os.path.join(PATH_PREFIX, 'metadata_labels.csv')
if os.path.exists(metadata_labels_path):
    metadata_labels = pd.read_csv(metadata_labels_path, index_col=0)
else:
    metadata_labels = None

# Define which column to use as the prediction
if 'Final_pred' in metadata_labels.columns:
    pred_col = 'Final_pred'
else:
    pred_col = 'Pred'

# Set the manually labelled with Prob = 1
metadata_labels.loc[metadata_labels['Prob'].isnull(), 'Prob'] = 1

# Select the NUM_SAMPLES_ORGANISE files with highest Prob that are labelled as `ap` and metadata said so too
metadata_labels = metadata_labels[(metadata_labels['Label'] == 'ap') & (metadata_labels[pred_col] == 'ap')].sort_values('Prob', ascending=False).iloc[:NUM_SAMPLES_ORGANISE]

In [ ]:
# Distribute equally the files on raw_folder which are contained on metadata_labels (labelled as `ap`)

relation_df, num_new_files = ORGANIZE_FOLDERs(raw_folder, ORGANIZE_FOLDER, relation_filepath, reset=False, groups=groups, subgroup_length=subgroup_length, filename_prefix=filename_prefix, force_extension='.dcm', copy=copy, metadata_labels=metadata_labels, debug=False)

In [ ]:
# Modify DICOM metadata of all the new files to rename patient info as the DICOM filename

# dicom_files = get_files(ORGANIZE_FOLDER, extensions='.dcm')
dicom_files = L(list(relation_df[-num_new_files:].apply(lambda row: Path(os.path.join(row.Path, row.Filename) + '.dcm'), axis=1).values))
rename_patient(dicom_files)

In [ ]:
# Generate all the missing templates on each folder

generate_template(ORGANIZE_FOLDER, groups, subgroup_length, filename_prefix=filename_prefix, excel=True, able_overwrite=able_overwrite)

In [24]:
modify_template(ORGANIZE_FOLDER, lambda df: df.drop('Side', axis=1), groups, subgroup_length, excel=True)

Folders: 100%|██████████| 87/87 [00:03<00:00, 25.47it/s]


In [ ]:
move_blocks(ORGANIZE_FOLDER, new_folder='Test_participant', blocks=[31,32,30,29], relation_filepath=relation_filepath, template_extension='xlsx')

In [ ]:
# Open relation file where the move/copy will be based on
relation_df = open_name_relation_file(relation_filepath, sep=',')

move_relation(relation_filepath, copy=True, to_raw=False)